In [1]:
import json
from os.path import join
import os
os.chdir("/home/tiagoalmeida/bioASQ-taskb/")

import sys
import pickle
import gc
import numpy as np
import tarfile
import random
from collections import defaultdict
from bisect import bisect
import math

##add keras to the modules
module_path = os.path.abspath(os.path.join('pubmed_data'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pubmed_data import pubmed_helper as ph

EMB_DIM = 200

import


## HAR

Reference PAPER:http://dmkd.cs.vt.edu/papers/WWW19.pdf

### Network structure
 - [General Network Configuration](#var_def)
 - [Self Attention Layer](#self_attn)
 - [Cross Attention Layer](#cross_attn)
 - [Final Network](#final_net)

In [2]:

#Load tokenizer and the embedding matrix

MODE = "regex_full_tokens"
tk = ph.load_tokenizer(mode=MODE)
emb_dict = ph.load_embeddings(mode=MODE)

assert len(tk.word_counts) == len(emb_dict)

#Number of different words
VOCAB_SIZE = len(tk.word_counts)+1

#Dimension of embeddings
EMB_DIM = emb_dict[1].shape[0]

emb_matrix = np.zeros((VOCAB_SIZE, EMB_DIM))

for i,vector in emb_dict.items():
    emb_matrix[i] = vector

Load regex_full_tokens_tokenizer.p
Load regex_full_tokens_word_embedding.p


<a id='var_def'></a>
## General Network Configuration

In [2]:

from tensorflow import unstack, stack
##Test 
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, activations
from tensorflow.keras.initializers import Zeros, Ones, Constant
from tensorflow.keras.layers import Dense, Lambda, Bidirectional, Dot,Masking,Reshape, Concatenate, Layer, Embedding, Input, Conv2D, GlobalMaxPooling2D, Flatten, TimeDistributed, GRU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.activations import tanh, sigmoid

from tensorflow.keras.preprocessing.sequence import pad_sequences


K.clear_session()

#Number max of term per query
MAX_Q_TERM = 13

#Number max of the snippet terms
MAX_SNIPPET_LENGTH = 13

MAX_NUMBER_SNIPPETS = 20

MAX_DOCUMENT_TOKENS = MAX_SNIPPET_LENGTH*MAX_NUMBER_SNIPPETS

ATTENTION_DIMENSION = 100

assert ATTENTION_DIMENSION%2==0

GRU_DIM = ATTENTION_DIMENSION//2

NUM_OF_SELF_ATTENTION = 1
#Train embedding weights
EMB_TRAINABLE = False

ACTIVATION_FUNCTION = "selu"




<a id='self_attn'></a>
### Self Attention Layer

In [7]:
class SelfAttention(Layer):
    
    def __init__(self, initializer='glorot_normal', regularizer=None, **kargs):
        super(SelfAttention, self).__init__(**kargs)

        self.initializer = initializer
        
        if regularizer is None or isinstance(regularizer,str):
            self.regularizer = regularizers.get(regularizer)
        else:
            self.regularizer = regularizer

    def build(self, input_shape):
        
        emb_dim = int(input_shape[2])

        self.W_attn_project = self.add_variable(name = "self_attention_projection",
                                   shape = [emb_dim, ATTENTION_DIMENSION],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)
        
        self.W_attn_score = self.add_variable(name = "self_attention_score",
                                   shape = [ATTENTION_DIMENSION, NUM_OF_SELF_ATTENTION],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)

        super(SelfAttention, self).build(input_shape)
    
    def call(self, x):
        print("x",x)
        #x_transpose = K.permute_dimensions(x,[0,2,1]) # (NONE, 100,15)
        #print("x_transpose",x_transpose)
        condition = K.all(x,keepdims=True,axis=-1)
        print("condition",condition)
        inv_condition = (1-K.cast(condition, K.floatx()))
        print("inv_condition",inv_condition)
        
        condition_all_zero = K.all(x,keepdims=True)
        print("condition_all_zero",condition_all_zero)
        inv_condition_all_zero = (1-K.cast(condition_all_zero, K.floatx()))
        print("inv_condition_all_zero",inv_condition_all_zero)
        
        
        x_projection = K.dot(x, self.W_attn_project) # (NONE, 300, 300)
        print("x_projection",x_projection)
        x_tanh = K.tanh(x_projection) # (NONE, 300, 15)
        print("x_tanh",x_tanh)
        x_attention = K.dot(x_tanh, self.W_attn_score)
        print("x_attention",x_attention)
        x_attention_maked = x_attention + (1.0 - inv_condition) * -100000.0
        print("x_attention_maked",x_attention_maked)
        #x_attention_squeeze = K.squeeze(x_score, axis=1)
        x_attention_softmax = K.softmax(x_attention_maked,axis = 1)
        print("x_attention_softmax",x_attention_softmax)
        x_attention_softmax_masked = x_attention_softmax * inv_condition_all_zero
        print("x_attention_softmax_masked",x_attention_softmax_masked)
        
        if (NUM_OF_SELF_ATTENTION>1):
            x_attention_softmax_transpose =  K.permute_dimensions(x_attention_softmax_masked,[0,2,1])
            print("x_attention_softmax_transpose",x_attention_softmax_transpose)

            x_attention_softmax_transpose_expand = K.expand_dims(x_attention_softmax_transpose)
            print("x_attention_softmax_transpose_expand",x_attention_softmax_transpose_expand)

            x_scored_emb = x_attention_softmax_transpose_expand * x
            print("x_scored_emb",x_scored_emb)
            x_attention_rep = K.sum(x_scored_emb, axis=2)
        else:
            x_scored_emb = x_attention_softmax_masked * x
            print("x_scored_emb",x_scored_emb)
            x_attention_rep = K.sum(x_scored_emb, axis=1)
            
        print("x_attention_rep",x_attention_rep)
        return x_attention_rep

In [8]:
#test self attention
visualize_self_attention = True
if visualize_self_attention:
    self_test = Sequential()
    self_test.add(Embedding(1000,300, input_length = MAX_Q_TERM))
    self_test.add(SelfAttention())

    self_test.summary()

x Tensor("embedding_2/embedding_lookup/Identity_1:0", shape=(?, 13, 300), dtype=float32)
condition Tensor("self_attention_2/All:0", shape=(?, 13, 1), dtype=bool)
inv_condition Tensor("self_attention_2/sub:0", shape=(?, 13, 1), dtype=float32)
condition_all_zero Tensor("self_attention_2/All_1:0", shape=(1, 1, 1), dtype=bool)
inv_condition_all_zero Tensor("self_attention_2/sub_1:0", shape=(1, 1, 1), dtype=float32)
x_projection Tensor("self_attention_2/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("self_attention_2/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("self_attention_2/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_maked Tensor("self_attention_2/add:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("self_attention_2/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax_masked Tensor("self_attention_2/mul_1:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("self_attention_2/mul_2:0", shape=(?, 13, 300

<a id='cross_attn'></a>
### Cross Attention Layer

In [6]:
class CrossAttention(Layer):
    
    def __init__(self, initializer='glorot_normal', regularizer=None, **kargs):
        super(CrossAttention, self).__init__(**kargs)

        self.initializer = initializer
        
        if regularizer is None or isinstance(regularizer,str):
            self.regularizer = regularizers.get(regularizer)
        else:
            self.regularizer = regularizer

    def build(self, input_shape):
        
        """
        input: [0] - query context embedding
               [1] - document context embedding
        """
        doc_embedding = input_shape[1]
        query_embedding = input_shape[0]
        
        self.query_len = query_embedding[1]
        self.doc_len = doc_embedding[1]
        
        print("query_len", self.query_len)
        print("doc_len", self.doc_len)
        
        assert int(query_embedding[2]) == int(doc_embedding[2])
        
        self.embedding_dim = int(query_embedding[2])
        
        self.W_sim_projection = self.add_variable(name = "similarity_projection",
                                   shape = [self.embedding_dim*3,1],
                                   initializer = self.initializer,
                                   regularizer = self.regularizer,)
        
        super(CrossAttention, self).build(input_shape)
    
    def call(self, x):
        """
        input: [0] - query context embedding
               [1] - document context embedding
        """
        doc_embedding = x[1]
        print("doc_embedding",doc_embedding.shape)
        query_embedding = x[0]
        print("query_embedding",query_embedding.shape)
        
        #build similarity matrix
        #row document token
        #colum query token
        doc_q_matrix = K.expand_dims(doc_embedding, axis=2)
        print("doc_q_matrix",doc_q_matrix.shape)
        doc_q_matrix = K.repeat_elements(doc_q_matrix, self.query_len, axis=2)
        print("doc_q_matrix",doc_q_matrix.shape)
        
        q_doc_matrix = K.expand_dims(query_embedding, axis=1)
        print("q_doc_matrix",q_doc_matrix.shape)
        q_doc_matrix = K.repeat_elements(q_doc_matrix, self.doc_len, axis=1)
        print("q_doc_matrix",q_doc_matrix.shape)
        
        element_mult = doc_q_matrix * q_doc_matrix
        print("element_mult",element_mult.shape)
        
        #concatenation
        S = K.concatenate([doc_q_matrix, q_doc_matrix, element_mult])
        print("S",S.shape)
        print("Wc",self.W_sim_projection.shape)
        S = K.dot(S, self.W_sim_projection)
        print("S",S.shape)
        
        S = K.squeeze(S,axis=-1)
        print("S",S.shape)
        
        S_D2Q = K.softmax(S, axis=1)
        print("S_D2Q",S_D2Q.shape)
        
        S_Q2D = K.softmax(S, axis=2)
        print("S_Q2D",S_Q2D.shape)
        
        A_D2Q = K.batch_dot(S_D2Q, query_embedding)
        print("A_D2Q",A_D2Q.shape)
        
        S_Q2D_transpose = K.permute_dimensions(S_Q2D,[0,2,1])
        print("S_Q2D_transpose",S_Q2D_transpose.shape)
        
        A_D2Q_Q2D = K.batch_dot(S_D2Q, S_Q2D_transpose)
        print("A_D2Q_Q2D",A_D2Q_Q2D.shape)
        
        A_Q2D = K.batch_dot(A_D2Q_Q2D, doc_embedding)
        print("A_Q2D",A_Q2D.shape)
        
        #concat
        doc_attn = doc_embedding * A_D2Q
        print("doc_attn",doc_attn.shape)
        doc_q_attn = doc_embedding * A_Q2D
        print("doc_q_attn",doc_q_attn.shape)
        
        V = K.concatenate([doc_embedding, A_D2Q, doc_attn, doc_q_attn])
        
        return V

In [7]:
#Test
visualize_cross_attention = False
if visualize_cross_attention:
    input_doc = Input(shape=(MAX_Q_TERM,))
    input_query = Input(shape=(MAX_SNIPPET_LENGTH,))

    emb_layer = Embedding(1000,300)

    cross_layer = CrossAttention()

    emb_doc = emb_layer(input_doc)
    emb_query = emb_layer(input_query)

    cross_attention = cross_layer([emb_query, emb_doc])

    model = Model(inputs=[input_query,input_doc], outputs = [cross_attention])
    model.summary()

<a id='final_net'></a>
### Final Network

In [8]:
#Layer Defenition

input_query = Input(shape=(MAX_Q_TERM,), name = "query_input")
input_doc = Input(shape=(MAX_NUMBER_SNIPPETS, MAX_SNIPPET_LENGTH), name = "document_input")
#input_doc = [Input(shape=(MAX_SNIPPET_LENGTH,), name = "document_input_"+j) for j in range(MAX_NUMBER_SNIPPETS)]

unstack_snippets = Lambda(lambda x:unstack(x,axis=1), name="unstack_snippets")

input_docs = unstack_snippets(input_doc)

emb_layer = Embedding(VOCAB_SIZE,EMB_DIM, weights=[emb_matrix],name="embedding_layer", trainable=EMB_TRAINABLE)

context_encoder = Bidirectional(GRU(GRU_DIM, return_sequences=True))

query_self_attention = SelfAttention()
cross_layer = CrossAttention()
snippet_lvl_1_self_attention = SelfAttention()
snippet_lvl_2_self_attention = SelfAttention()

snippet_expand_dim = Lambda(lambda x:K.expand_dims(x,axis=1))
snippet_concat = Lambda(lambda x:K.concatenate(x,axis=1))

snippets_projection = Dense(ATTENTION_DIMENSION)

q_doc_mult = Lambda(lambda x:x[0]*x[1])

#NO ACTIVATION??
fnn_h1 = Dense(ATTENTION_DIMENSION)
fnn_h2 = Dense(ATTENTION_DIMENSION//2)
fnn_h3 = Dense(1)

"""
ASSEMBLE
"""

query_emb = emb_layer(input_query)
query_context_aware = context_encoder(query_emb)
query_weighted = query_self_attention(query_context_aware)

snippets = []
for input_snippet in input_docs:
    snippet_emb = emb_layer(input_snippet)
    snippet_context_aware = context_encoder(snippet_emb)
    snippet_query = cross_layer([query_context_aware, snippet_context_aware])
    snippet_weighted = snippet_lvl_1_self_attention(snippet_query)
    snippets.append(snippet_expand_dim(snippet_weighted))

snippets = snippet_concat(snippets)
snippets = snippet_lvl_2_self_attention(snippets)
snippets = snippets_projection(snippets)
q_snippets_rep = q_doc_mult([query_weighted, snippets])
score = fnn_h1(q_snippets_rep)
score = fnn_h2(score)
score = fnn_h3(score)

inputs = [input_query, input_doc]

har_model = Model(inputs=inputs, outputs = [score])

har_model.summary()

x_projection Tensor("self_attention/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("self_attention/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("self_attention/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("self_attention/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("self_attention/mul:0", shape=(?, 13, 100), dtype=float32)
x_attention_rep Tensor("self_attention/Sum:0", shape=(?, 100), dtype=float32)
query_len 13
doc_len 13
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
x_projection Tensor("self_attention_1/Reshape_2:0"

doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
x_projection Tensor("self_attention_1_9/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("self_attention_1_9/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("self_attention_1_9/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("self_attention_1_9/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("self_attention_1_9/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("self_attention_1_9/Sum:0", shape=(?, 400), dtype=float32)
doc_embedding (?, 13, 100)
query_embedding (?, 13,

doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
x_projection Tensor("self_attention_1_18/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("self_attention_1_18/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("self_attention_1_18/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("self_attention_1_18/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("self_attention_1_18/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("self_attention_1_18/Sum:0", shape=(?, 400), dtype=float32)
doc_embedding (?, 13, 100)
query_embedding (

## Build training Arch

In [9]:
# total number of negative samples

NUM_NEG_SAMPLES = 6
TOTAL_NUM_SAMPLES = NUM_NEG_SAMPLES+1
#input

input_query = Input(shape=(MAX_Q_TERM,))
input_docs = Input(shape=(TOTAL_NUM_SAMPLES, MAX_NUMBER_SNIPPETS, MAX_SNIPPET_LENGTH,))

unstack_documents = Lambda(lambda x:unstack(x,axis=1), name="unstack_documents")

list_of_documents = unstack_documents(input_docs)
print("list_of_documents[0]",list_of_documents[0])
scores = []
for doc_tensor in list_of_documents:
    print("input_query", input_query)
    print("doc_tensor", doc_tensor)
    scores.append(har_model([input_query, doc_tensor]))
    
har_trainable_model = Model(inputs=[input_query, input_docs], outputs=scores)

#write tensor operation loss function


pos_score = scores[0]
maximum = [K.maximum(0.0,1.0 - pos_score + neg_score) for neg_score in scores[1:]]
concat = K.concatenate(maximum)
print("concat",concat)

loss = K.mean(K.sum(concat,axis=-1))

print("loss",concat)

    
har_trainable_model.add_loss(loss)


har_trainable_model.summary()



list_of_documents[0] Tensor("unstack_documents/unstack:0", shape=(?, 20, 13), dtype=float32)
input_query Tensor("input_1:0", shape=(?, 13), dtype=float32)
doc_tensor Tensor("unstack_documents/unstack:0", shape=(?, 20, 13), dtype=float32)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D

x_tanh Tensor("model/self_attention_1/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model/self_attention_1/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model/self_attention_1/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/self_attention_1/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/self_attention_1/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model/self_attention_1_1/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/self_attention_1_1/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model/self_attention_1_1/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model/self_attention_1_1/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/self_attention_1_1/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/self_attention_1_1/Sum:0", shape=(?, 400), dtype=float32)
x_pro

x_attention_softmax Tensor("model/self_attention_1_17/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/self_attention_1_17/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/self_attention_1_17/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model/self_attention_1_18/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/self_attention_1_18/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model/self_attention_1_18/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model/self_attention_1_18/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model/self_attention_1_18/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model/self_attention_1_18/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model/self_attention_1_19/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model/self_attention_1_19/Tanh:0", shape=(?, 13, 10

S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_

x_attention Tensor("model_1/self_attention_1_13/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_1/self_attention_1_13/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_1/self_attention_1_13/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_1/self_attention_1_13/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_1/self_attention_1_14/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_1/self_attention_1_14/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_1/self_attention_1_14/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_1/self_attention_1_14/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_1/self_attention_1_14/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_1/self_attention_1_14/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_1/self_attention_1_15/

...

doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mu

x_projection Tensor("model_5/self_attention_1_1/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_5/self_attention_1_1/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_5/self_attention_1_1/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_5/self_attention_1_1/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_5/self_attention_1_1/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_5/self_attention_1_1/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_5/self_attention_1_2/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_5/self_attention_1_2/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_5/self_attention_1_2/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_5/self_attention_1_2/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_5/self_attention_1_2/mul:0", s

x_attention_softmax Tensor("model_5/self_attention_1_18/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_5/self_attention_1_18/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_5/self_attention_1_18/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_5/self_attention_1_19/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_5/self_attention_1_19/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_5/self_attention_1_19/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_5/self_attention_1_19/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_5/self_attention_1_19/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_5/self_attention_1_19/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_5/self_attention_2/Reshape_2:0", shape=(?, 20, 100), dtype=float32)
x_tanh Tensor("model_5/self_attention_2/Tanh:0", 

q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13, 13, 1)
S (?, 13, 13)
S_D2Q (?, 13, 13)
S_Q2D (?, 13, 13)
A_D2Q (?, 13, 100)
S_Q2D_transpose (?, 13, 13)
A_D2Q_Q2D (?, 13, 13)
A_Q2D (?, 13, 100)
doc_attn (?, 13, 100)
doc_q_attn (?, 13, 100)
doc_embedding (?, 13, 100)
query_embedding (?, 13, 100)
doc_q_matrix (?, 13, 1, 100)
doc_q_matrix (?, 13, 13, 100)
q_doc_matrix (?, 1, 13, 100)
q_doc_matrix (?, 13, 13, 100)
element_mult (?, 13, 13, 100)
S (?, 13, 13, 300)
Wc (300, 1)
S (?, 13

x_attention_softmax Tensor("model_6/self_attention_1_12/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_6/self_attention_1_12/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_6/self_attention_1_12/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_6/self_attention_1_13/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_6/self_attention_1_13/Tanh:0", shape=(?, 13, 100), dtype=float32)
x_attention Tensor("model_6/self_attention_1_13/Reshape_5:0", shape=(?, 13, 1), dtype=float32)
x_attention_softmax Tensor("model_6/self_attention_1_13/transpose_3:0", shape=(?, 13, 1), dtype=float32)
x_scored_emb Tensor("model_6/self_attention_1_13/mul:0", shape=(?, 13, 400), dtype=float32)
x_attention_rep Tensor("model_6/self_attention_1_13/Sum:0", shape=(?, 400), dtype=float32)
x_projection Tensor("model_6/self_attention_1_14/Reshape_2:0", shape=(?, 13, 100), dtype=float32)
x_tanh Tensor("model_6/self_attention_1_14/Tan

In [10]:
WRITE_GRAPH = False
if WRITE_GRAPH:
    from tensorflow.summary import FileWriter

    graph = K.get_session().graph
     # Your model implementation
    #with graph.as_default():
      # compile method actually creates the model in the graph.
      #deepRank_model.compile(loss=identity_loss, optimizer='adam', metrics=['accuracy'])
    writer = FileWriter(logdir='tensorboard/har', graph=graph)
    writer.flush()



## Set up training data

In [11]:
path_dl_train = "/backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz"



tar = tarfile.open(path_dl_train)
#open
print("Open",path_dl_train)
m = tar.getmembers()[0]
f = tar.extractfile(m)
train_articles_collection = pickle.load(f)

f.close()

Open /backup/results/fast_method_relevant_results/train_data_deep_models_v2.tar.gz


In [12]:
PARTILLY_POSITIVE_SAMPLES = 2
NEGATIVE_SAMPLES = 4

class TrainDataGenerator(object):
    def __init__(self, article_collection, tokenizer, batch_queries_size):
        
        self.batch_size = batch_queries_size
        self.tokenizer = tokenizer
        
        self.train_data = article_collection["bioasq_data"]
        self.articles = article_collection["collection"]
        self.irrelevant_pmid = article_collection["irrelevant_pmid"]
        
        self.num_steps = len(self.train_data)//self.batch_size
    
    def __len__(self):
        return self.num_steps
    
    def __iter__(self):
        
        
        query = []
        query_doc = [] #(pos,neg,...,neg)
        query_negative_doc = []

        
        while True:
            
            #stop condition
            if len(query)>=self.batch_size:
                #missing fill the gap for the missing query_terms
                query = np.array(query)
                #p=np.random.permutation(query.shape[0])
                #query = query[p]
                query_doc = np.array(query_doc)#[p]

                
                X = [query, query_doc]
                #Y = [np.zeros((len(query))),np.zeros((len(query)))]
                yield X


                #reset
                query = []
                query_doc = []
                query_negative_doc = []
            
            docs = []
            
            #select a random question
            random_query_index = random.randint(0, len(self.train_data)-1) 
            query_data = self.train_data[random_query_index]
            
            #list of partilly relevant documents
            partilly_positive_pmid_docs = query_data["partilly_positive_pmid"]
            tokenized_query = query_data["query"][:MAX_Q_TERM]
            
            query.append(tokenized_query)
            
            #positive doc
            #select a random positive
            random_doc_index = random.randint(0, len(query_data["positive_pmid"])-1) 
            doc_pmid = query_data["positive_pmid"][random_doc_index]

            tokenized_positive_doc = self.articles[doc_pmid]
            positive_snippets = self.__snippet_split(tokenized_positive_doc)
            
            docs.append(positive_snippets)
            
            for j in range(PARTILLY_POSITIVE_SAMPLES+NEGATIVE_SAMPLES):
                
                
                if j<PARTILLY_POSITIVE_SAMPLES:
                    #select the partilly posivite doc
                    random_ind = bisect(query_data["partially_positive_cumulative_prob"],random.random())
                    random_negative_doc_pmid = query_data["partilly_positive_pmid"][random_ind]
                    #print(self.__get_article(random_negative_doc_pmid))
                    tokenized_negative_doc = self.articles[random_negative_doc_pmid]
                    negative_snippets = self.__snippet_split( tokenized_negative_doc)
                else:
                    #select a random negative
                    random_doc_index = random.randint(0, len(self.irrelevant_pmid)-1) 
                    doc_pmid = self.irrelevant_pmid[random_doc_index]
                    
                    tokenized_negative_doc = self.articles[doc_pmid]
                    negative_snippets = self.__snippet_split( tokenized_negative_doc)
                
                #negative doc
                docs.append(negative_snippets)

            query_doc.append(docs)

            
    def __snippet_split(self, tokenized_doc, snippet_length=MAX_SNIPPET_LENGTH):
        

        if len(tokenized_doc) < MAX_DOCUMENT_TOKENS:
            #pad
            tokenized_doc += [0]*(MAX_DOCUMENT_TOKENS-len(tokenized_doc))
            
        else:
            tokenized_doc = tokenized_doc[:MAX_DOCUMENT_TOKENS] #cut
        
        index_list = list(range(0, MAX_DOCUMENT_TOKENS, MAX_SNIPPET_LENGTH))+[MAX_DOCUMENT_TOKENS]
        #print(tokenized_doc)
        snippets = [ tokenized_doc[index_list[i]:index_list[i+1]] for i in range(len(index_list)-1)  ]
        #print(snippets)
        return snippets
        
        

In [13]:
data_generator = TrainDataGenerator(train_articles_collection, tk, 128)
data_generator = iter(data_generator)
X = next(data_generator)

In [14]:
X[0].shape

(128, 13)

## Test (validation) data

In [15]:
path_dl_test = "/backup/results/fast_method_relevant_results/test_data_deep_models_v2.tar.gz"



tar = tarfile.open(path_dl_test)
#open
print("Open",path_dl_test)
m = tar.getmembers()[0]
f = tar.extractfile(m)
test_articles_collection = pickle.load(f)

f.close()

Open /backup/results/fast_method_relevant_results/test_data_deep_models_v2.tar.gz


In [16]:
class TestDataGenerator(object):
    def __init__(self, article_collection, tokenizer):
        
        self.tokenizer = tokenizer
        
        self.test_data = article_collection["bioasq_data"] 
        self.articles = article_collection["collection"]
        
        self.num_steps = len(self.test_data)
        
    
    def __get_article(self, pmid):
        return self.article_map(self.articles[pmid])
    
    def __len__(self):
        return self.num_steps
    
    def __iter__(self):
        
        
        query = []
        query_doc = []
        

        for query_data in self.test_data:


            tokenized_query = query_data["query"][:MAX_Q_TERM]

            for doc_pmid in query_data["documents"]:

                tokenized_doc = self.articles[doc_pmid]
                doc_snippets = self.__snippet_split(tokenized_doc)

                ### add ###
                query.append(tokenized_query)

                #positive doc
                query_doc.append(doc_snippets)


            #missing fill the gap for the missing query_terms

            X = [np.array(query), np.array(query_doc)]

            yield X

            #reset
            query = []
            query_doc = []

                
    def __snippet_split(self, tokenized_doc, snippet_length=MAX_SNIPPET_LENGTH):
        

        if len(tokenized_doc) < MAX_DOCUMENT_TOKENS:
            #pad
            tokenized_doc += [0]*(MAX_DOCUMENT_TOKENS-len(tokenized_doc))
            
        else:
            tokenized_doc = tokenized_doc[:MAX_DOCUMENT_TOKENS] #cut
        
        index_list = list(range(0, MAX_DOCUMENT_TOKENS, MAX_SNIPPET_LENGTH))+[MAX_DOCUMENT_TOKENS]
        #print(tokenized_doc)
        snippets = [ tokenized_doc[index_list[i]:index_list[i+1]] for i in range(len(index_list)-1)  ]
        #print(snippets)
        return snippets

In [17]:
validation_articles_collection = {"bioasq_data":[],"collection":test_articles_collection["collection"]}

_temp = test_articles_collection["bioasq_data"][:]
random.shuffle(_temp)
print(len(_temp))

validation_percentage = 0.15

split_index = int(len(_temp)*validation_percentage)
print(split_index)

validation_articles_collection["bioasq_data"] = _temp[:split_index]
#test_articles_collection["bioasq_data"] = _temp[split_index:]

print("validation size",len(validation_articles_collection["bioasq_data"]),"test size",len(test_articles_collection["bioasq_data"]))

549
82
validation size 82 test size 549


In [18]:
def validate_test_data(data):
    data_generator = TestDataGenerator(data, tk)
    data_generator = iter(data_generator)

    query_results = {}

    for i,X in enumerate(data_generator):
        print("Predict query:",i,end="\r")
        har_ranking = har_model.predict(X)
        har_ranking = map(lambda x:x[0],har_ranking.tolist())
        bm25_results = data["bioasq_data"][i]["documents"]
        har_ranking_pmid = list(zip(bm25_results,har_ranking))
        har_ranking_pmid.sort(key=lambda x:-x[1])
        query_results[data["bioasq_data"][i]["id"]] = {"result":har_ranking_pmid,"goldstandard":data["bioasq_data"][i]["positive_pmid"]}
        #print("save query results:",i,end="\r")
        
    return query_results

In [19]:
from models.generic_model import ModelAPI, f_recall, f_map

def validation_score(deep_rank_test_query_results):
    id_to_remove = []
    for k,v in deep_rank_test_query_results.items():
        if len(v["goldstandard"]) == 0:
            id_to_remove.append(k)

    for k in id_to_remove:
        del deep_rank_test_query_results[k]

    deep_rank_test_query_results = list(deep_rank_test_query_results.values())

    print("TEST set, len ",len(deep_rank_test_query_results))

    expectations = list(map(lambda x:x["goldstandard"],deep_rank_test_query_results))
    predictions = list(map(lambda x:x["result"],deep_rank_test_query_results))

    #print("Recall:",f_recall(predictions,expectations,at=1000))
    bioasq_map = f_map(predictions,expectations,bioASQ=True)
    print("MAP @10 bioASQ:", bioasq_map)
    print("MAP @25:",f_map(predictions,expectations, bioASQ=True, at=25))
    print("MAP @50:",f_map(predictions,expectations, bioASQ=True, at=50))
    print("MAP @100:",f_map(predictions,expectations, bioASQ=True, at=100))
    print("MAP @200:",f_map(predictions,expectations, bioASQ=True, at=200))
    print("MAP @300:",f_map(predictions,expectations, bioASQ=True, at=300))

    print("MAP:",f_map(predictions,expectations, use_len=True))
    
    print("RECALL@10:",f_recall(predictions,expectations, at=10))
    print("RECALL@50:",f_recall(predictions,expectations, at=50))
    print("RECALL@100:",f_recall(predictions,expectations, at=100))
    return bioasq_map, predictions, expectations

In [20]:
from tensorflow.keras.optimizers import SGD, Adam,  Adadelta

#sgd = SGD(lr=0.001)
#adam = Adam(lr=0.001)
adadelta = Adadelta(lr=2)

har_trainable_model.compile(optimizer=adadelta)

In [21]:
#validation = validate_test_data(validation_articles_collection)

In [22]:
#_ = validation_score(validation)

In [32]:
gen = TrainDataGenerator(train_articles_collection, tk, 256)

gen_iter = iter(gen)

#loss = []

for i,line in enumerate(loss):
    print("Epoach:",i,"| avg loss:",np.mean(loss[i]),"| max loss:",np.max(loss[i]),"| min loss:",np.min(loss[i]))

import time

max_bio_map_val = 0.10
max_bio_map_test = 0

for epoach in range(950,1000):
    loss_per_epoach = []
    for step in range(len(gen)):
        X = next(gen_iter)
        
        start = time.time()
        loss_per_epoach.append(har_trainable_model.train_on_batch(X))
        print("Step:",step,"| loss:",loss_per_epoach[-1],"| current max loss:",np.max(loss_per_epoach),"| current min loss:",np.min(loss_per_epoach),"| time:",time.time()-start,end="\r")
        

    
    
    if epoach%20==0:
        print("")
        validate_query_results = validate_test_data(validation_articles_collection)
        print("")
        bio_map_val,_,_ = validation_score(validate_query_results)
        _ = validation_score(validate_test_data(validation_train_articles_collection))
        
        if bio_map_val >= max_bio_map_val:
            max_bio_map_val = bio_map_val
            print("")
            print("Run for the test set")
            test_query_results = validate_test_data(test_articles_collection)
            bio_map_test = validation_score(test_query_results)
            
            if bio_map_test >= max_bio_map_test:
                max_bio_map_test = bio_map_test
                
                har_model.save_weights("har_weights.h5")
                #har_model.save("har_model.h5")
                
    loss.append(loss_per_epoach)
    print("",end="\r")#clear the line
    print("Epoach:",epoach,"| avg loss:",np.mean(loss[-1]),"| max loss:",np.max(loss[-1]),"| min loss:",np.min(loss[-1]))

Epoach: 0 | avg loss: 5.9780874 | max loss: 5.99463 | min loss: 5.9432344
Epoach: 1 | avg loss: 4.905095 | max loss: 5.8984766 | min loss: 3.7576256
Epoach: 2 | avg loss: 2.9906387 | max loss: 3.3897157 | min loss: 2.704164
Epoach: 3 | avg loss: 2.8194482 | max loss: 2.949489 | min loss: 2.4600127
Epoach: 4 | avg loss: 2.628375 | max loss: 2.8297503 | min loss: 2.4962373
Epoach: 5 | avg loss: 2.685218 | max loss: 3.0441256 | min loss: 2.4424398
Epoach: 6 | avg loss: 2.5781336 | max loss: 2.8284454 | min loss: 2.2068458
Epoach: 7 | avg loss: 2.6436124 | max loss: 2.9304729 | min loss: 2.389176
Epoach: 8 | avg loss: 2.4605103 | max loss: 2.64789 | min loss: 2.194899
Epoach: 9 | avg loss: 2.491694 | max loss: 2.780827 | min loss: 2.0749748
Epoach: 10 | avg loss: 2.4773917 | max loss: 2.5970063 | min loss: 2.3449244
Epoach: 11 | avg loss: 2.595004 | max loss: 2.7311952 | min loss: 2.4138684
Epoach: 12 | avg loss: 2.4845245 | max loss: 2.8401804 | min loss: 2.1962109
Epoach: 13 | avg loss: 

Epoach: 243 | avg loss: 0.40735593 | max loss: 0.5042627 | min loss: 0.31701452
Epoach: 244 | avg loss: 0.38371906 | max loss: 0.47881553 | min loss: 0.3237277
Epoach: 245 | avg loss: 0.41004854 | max loss: 0.51557785 | min loss: 0.3346104
Epoach: 246 | avg loss: 0.38782218 | max loss: 0.45546472 | min loss: 0.29507425
Epoach: 247 | avg loss: 0.3810554 | max loss: 0.47742644 | min loss: 0.2977942
Epoach: 248 | avg loss: 0.387599 | max loss: 0.4583307 | min loss: 0.3143763
Epoach: 249 | avg loss: 0.36149937 | max loss: 0.40152442 | min loss: 0.30163467
Epoach: 250 | avg loss: 0.37099195 | max loss: 0.43509474 | min loss: 0.30106178
Epoach: 251 | avg loss: 0.35733464 | max loss: 0.40697438 | min loss: 0.3056534
Epoach: 252 | avg loss: 0.41914988 | max loss: 0.5357771 | min loss: 0.3324978
Epoach: 253 | avg loss: 0.38033384 | max loss: 0.44137362 | min loss: 0.30842757
Epoach: 254 | avg loss: 0.39263186 | max loss: 0.43971306 | min loss: 0.30840763
Epoach: 255 | avg loss: 0.3638817 | max 

Epoach: 493 | avg loss: 0.25434887 | max loss: 0.29023743 | min loss: 0.21306252
Epoach: 494 | avg loss: 0.23801884 | max loss: 0.3016582 | min loss: 0.17613351
Epoach: 495 | avg loss: 0.22312634 | max loss: 0.28792608 | min loss: 0.16094565
Epoach: 496 | avg loss: 0.25431412 | max loss: 0.35450387 | min loss: 0.16501051
Epoach: 497 | avg loss: 0.20831385 | max loss: 0.29539222 | min loss: 0.16311947
Epoach: 498 | avg loss: 0.25394672 | max loss: 0.2960355 | min loss: 0.1797258
Epoach: 499 | avg loss: 0.26470274 | max loss: 0.34599465 | min loss: 0.18545547
Epoach: 500 | avg loss: 0.22826105 | max loss: 0.3097149 | min loss: 0.18781382
Epoach: 501 | avg loss: 0.22862191 | max loss: 0.30208138 | min loss: 0.1372377
Epoach: 502 | avg loss: 0.2519961 | max loss: 0.38351086 | min loss: 0.19869298
Epoach: 503 | avg loss: 0.27166843 | max loss: 0.32903162 | min loss: 0.20956878
Epoach: 504 | avg loss: 0.27050254 | max loss: 0.32535392 | min loss: 0.21622923
Epoach: 505 | avg loss: 0.24580842

Epoach: 743 | avg loss: 0.17539416 | max loss: 0.2122785 | min loss: 0.12786412
Epoach: 744 | avg loss: 0.16023585 | max loss: 0.19707641 | min loss: 0.115837656
Epoach: 745 | avg loss: 0.1799075 | max loss: 0.2668154 | min loss: 0.11339701
Epoach: 746 | avg loss: 0.18956819 | max loss: 0.20785676 | min loss: 0.15741517
Epoach: 747 | avg loss: 0.1672079 | max loss: 0.19771615 | min loss: 0.114578985
Epoach: 748 | avg loss: 0.19373025 | max loss: 0.24427074 | min loss: 0.14855902
Epoach: 749 | avg loss: 0.20005928 | max loss: 0.29413846 | min loss: 0.15801251
Epoach: 750 | avg loss: 0.21554911 | max loss: 0.2824101 | min loss: 0.124118894
Epoach: 751 | avg loss: 0.18884698 | max loss: 0.2525644 | min loss: 0.122447826
Epoach: 752 | avg loss: 0.17703968 | max loss: 0.21971366 | min loss: 0.11433394
Epoach: 753 | avg loss: 0.18385942 | max loss: 0.23219936 | min loss: 0.11878079
Epoach: 754 | avg loss: 0.18037857 | max loss: 0.23499517 | min loss: 0.12782644
Epoach: 755 | avg loss: 0.1847

Epoach: 950 | avg loss: 0.16267064 | max loss: 0.2000409 | min loss: 0.136145163614516 | time: 11.04591703414917152
Epoach: 951 | avg loss: 0.15244317 | max loss: 0.20265019 | min loss: 0.11175933511759335 | time: 11.0131354331970211
Epoach: 952 | avg loss: 0.13572313 | max loss: 0.1899216 | min loss: 0.1068796110687961 | time: 11.0069119930267339
Epoach: 953 | avg loss: 0.16409722 | max loss: 0.23220104 | min loss: 0.1020675710206757 | time: 11.037168979644775
Epoach: 954 | avg loss: 0.1309264 | max loss: 0.16313866 | min loss: 0.11155788611557886 | time: 11.00368738174438514
Epoach: 955 | avg loss: 0.15871972 | max loss: 0.18337372 | min loss: 0.136174383617438 | time: 11.0145530700683637
Epoach: 956 | avg loss: 0.15214795 | max loss: 0.28397435 | min loss: 0.0710054607100546 | time: 11.029855012893677
Epoach: 957 | avg loss: 0.15109581 | max loss: 0.17666426 | min loss: 0.1295439.1295439 | time: 11.0759370326995851
Epoach: 958 | avg loss: 0.14123742 | max loss: 0.1852453 | min loss:

In [26]:
validation_train_articles_collection = {"bioasq_data":[],"collection":train_articles_collection["collection"]}

_temp = train_articles_collection["bioasq_data"][:]
random.shuffle(_temp)
print(len(_temp))

validation_percentage = 0.04

split_index = int(len(_temp)*validation_percentage)
print(split_index)

validation_train_articles_collection["bioasq_data"] = _temp[:split_index]
#test_articles_collection["bioasq_data"] = _temp[split_index:]

print("validation train size",len(validation_train_articles_collection["bioasq_data"]))

del _temp

2137
85
validation train size 85


In [36]:
test = validate_test_data(test_articles_collection)
_test_validation_scores = validation_score(test)

TEST set, len  542
MAP @10 bioASQ: 0.04496676038188954
MAP @25: 0.0634085317578492
MAP @50: 0.07977501611963189
MAP @100: 0.09286495409665439
MAP @200: 0.10269169701889902
MAP @300: 0.10747185573911922
MAP: 0.077011123930585
RECALL@10: 0.1441388080767276
RECALL@50: 0.34713683151100727
RECALL@100: 0.474338002391366


In [37]:
_train = validate_test_data(train_articles_collection)
_train_validation_scores = validation_score(_train)

TEST set, len  2137
MAP @10 bioASQ: 0.17697686639778398
MAP @25: 0.25510361756180516
MAP @50: 0.30344370648908725
MAP @100: 0.33134215979994863
MAP @200: 0.3429918827961693
MAP @300: 0.34565611336998997
MAP: 0.32187774858350454
RECALL@10: 0.5204272246827295
RECALL@50: 0.8622614773925681
RECALL@100: 0.9447249088296876


In [40]:
har_model.save_weights("har_model_weights.h5")